<a href="https://colab.research.google.com/github/elangbijak4/NLP-AI-Algorithm/blob/main/Demo_Melatih_SVM_untuk_Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn beautifulsoup4 requests

In [2]:
import requests
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [11]:
# Fungsi untuk melakukan web scraping dan mengambil teks dari halaman web
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Misalkan kita mengambil beberapa paragraf dari tag <p> di halaman web
    paragraphs = soup.find_all('p')[:5]  # Ambil lima paragraf pertama sebagai contoh
    text = ' '.join([p.get_text() for p in paragraphs])
    return text

# Fungsi untuk membersihkan teks
def clean_text(text):
    # Hapus karakter non-alfanumerik
    text = re.sub(r'\W', ' ', text)
    # Hapus karakter tunggal
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Hapus beberapa spasi
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Ubah teks menjadi huruf kecil
    text = text.lower()
    return text

# URL untuk di-scrapping
url = 'https://kompas.com'
text = scrape_website(url)
cleaned_text = clean_text(text)

# Contoh dataset hasil web scraping (misalnya review)
corpus = [
    {'text': cleaned_text, 'sentiment': 'positive'},
    # Tambahkan lebih banyak data dari hasil scraping jika diperlukan
]

# Menambahkan lebih banyak sampel teks ke corpus (misalnya 100 sampel)
for i in range(1, 101):
    # Variasikan sentiment untuk beberapa sampel
    sentiment = 'positive' if i % 2 == 0 else 'negative'
    corpus.append({'text': f'Example text {i}', 'sentiment': sentiment})  # Ganti dengan teks dari hasil scraping

In [12]:
corpus

[{'text': 'news 2 menit lalu edukasi 4 menit lalu regional 6 menit lalu regional 6 menit lalu money 7 menit lalu',
  'sentiment': 'positive'},
 {'text': 'Example text 1', 'sentiment': 'negative'},
 {'text': 'Example text 2', 'sentiment': 'positive'},
 {'text': 'Example text 3', 'sentiment': 'negative'},
 {'text': 'Example text 4', 'sentiment': 'positive'},
 {'text': 'Example text 5', 'sentiment': 'negative'},
 {'text': 'Example text 6', 'sentiment': 'positive'},
 {'text': 'Example text 7', 'sentiment': 'negative'},
 {'text': 'Example text 8', 'sentiment': 'positive'},
 {'text': 'Example text 9', 'sentiment': 'negative'},
 {'text': 'Example text 10', 'sentiment': 'positive'},
 {'text': 'Example text 11', 'sentiment': 'negative'},
 {'text': 'Example text 12', 'sentiment': 'positive'},
 {'text': 'Example text 13', 'sentiment': 'negative'},
 {'text': 'Example text 14', 'sentiment': 'positive'},
 {'text': 'Example text 15', 'sentiment': 'negative'},
 {'text': 'Example text 16', 'sentiment':

In [13]:
# Konversi dataset menjadi DataFrame atau gunakan secara langsung tergantung pada struktur data

# Proses dataset untuk SVM
X = [entry['text'] for entry in corpus]
y = [entry['sentiment'] for entry in corpus]

# Label Encoding untuk sentimen
le = LabelEncoder()
y = le.fit_transform(y)

# Check if you have enough samples before splitting
if len(X) > 1:
    # Pembagian dataset menjadi data latih dan data uji
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Pipeline untuk SVM dengan TF-IDF sebagai ekstraksi fitur
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SVC(kernel='linear')),
    ])

    # Grid search untuk optimisasi hiperparameter
    parameters = {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'tfidf__use_idf': (True, False),
        'clf__C': [0.1, 1, 10],
    }

    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)

    # Prediksi dengan model terbaik
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Evaluasi model
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, target_names=le.classes_))
else:
    print("Dataset does not have enough samples to split. Add more data.")
    # Collect more data for your corpus here

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Accuracy: 0.42857142857142855
              precision    recall  f1-score   support

    negative       0.43      1.00      0.60         9
    positive       0.00      0.00      0.00        12

    accuracy                           0.43        21
   macro avg       0.21      0.50      0.30        21
weighted avg       0.18      0.43      0.26        21



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
